# Tensor的基本操作

## 算术操作

在PyTorch中，同一种操作可能有很多种形式，下面用加法作为例子。

In [1]:
import torch
# 加法形式1
x = torch.rand(5, 3)
y = torch.rand(5, 3)
print(x + y)

# 加法形式2
print(torch.add(x, y))

# 加法形式3
# adds x to y
y.add_(x)
print(y)

tensor([[0.2920, 0.9187, 0.5465],
        [0.8588, 0.5609, 1.6790],
        [0.6618, 1.2261, 0.7827],
        [0.7373, 0.8956, 0.5042],
        [1.6560, 0.7342, 0.4946]])
tensor([[0.2920, 0.9187, 0.5465],
        [0.8588, 0.5609, 1.6790],
        [0.6618, 1.2261, 0.7827],
        [0.7373, 0.8956, 0.5042],
        [1.6560, 0.7342, 0.4946]])
tensor([[0.2920, 0.9187, 0.5465],
        [0.8588, 0.5609, 1.6790],
        [0.6618, 1.2261, 0.7827],
        [0.7373, 0.8956, 0.5042],
        [1.6560, 0.7342, 0.4946]])


> 注：PyTorch操作inplace版本都有后缀_, 例如x.copy_(y), x.t_()

## 索引

我们还可以使用类似NumPy的索引操作来访问Tensor的一部分，需要注意的是：索引出来的结果与原数据共享内存，也即修改一个，另一个会跟着修改。

In [2]:
y = x[0, :]
y += 1
print(y)
print(x[0, :]) # 源tensor也被改了

tensor([1.2380, 1.2546, 1.1699])
tensor([1.2380, 1.2546, 1.1699])


除了常用的索引选择数据之外，PyTorch还提供了一些高级的选择函数:

|函数	|功能|
|--|--|
|index_select(input, dim, index)	|在指定维度dim上选取，比如选取某些行、某些列|
|masked_select(input, mask)	|例子如上，a[a>0]，使用ByteTensor进行选取|
|nonzero(input)	|非0元素的下标|
|gather(input, dim, index)	|根据index，在dim维度上选取数据，输出的size与index一样|

更多操作可参阅:https://pytorch.org/docs/stable/tensors.html?highlight=tensor#torch.Tensor

## 改变形状

用view()来改变Tensor的形状：

In [3]:
y = x.view(15)
z = x.view(-1, 5)  # -1所指的维度可以根据其他维度的值推出来
print(x.size(), y.size(), z.size())

torch.Size([5, 3]) torch.Size([15]) torch.Size([3, 5])


注意view()返回的新Tensor与源Tensor虽然可能有不同的size，但是是共享data的，也即更改其中的一个，另外一个也会跟着改变。(顾名思义，view仅仅是改变了对这个张量的观察角度，内部数据并未改变)

In [4]:
x += 1
print(x)
print(y) # 也加了1

tensor([[2.2380, 2.2546, 2.1699],
        [1.8353, 1.0111, 1.8369],
        [1.6252, 1.3025, 1.4373],
        [1.0699, 1.8489, 1.0358],
        [1.9720, 1.5125, 1.1718]])
tensor([2.2380, 2.2546, 2.1699, 1.8353, 1.0111, 1.8369, 1.6252, 1.3025, 1.4373,
        1.0699, 1.8489, 1.0358, 1.9720, 1.5125, 1.1718])


所以如果我们想返回一个真正新的副本（即不共享data内存）该怎么办呢？Pytorch还提供了一个reshape()可以改变形状，但是此函数并不能保证返回的是其拷贝，所以不推荐使用。推荐先用clone创造一个副本然后再使用view。

In [5]:
x_cp = x.clone().view(15)
x -= 1
print(x)
print(x_cp)

tensor([[1.2380, 1.2546, 1.1699],
        [0.8353, 0.0111, 0.8369],
        [0.6252, 0.3025, 0.4373],
        [0.0699, 0.8489, 0.0358],
        [0.9720, 0.5125, 0.1718]])
tensor([2.2380, 2.2546, 2.1699, 1.8353, 1.0111, 1.8369, 1.6252, 1.3025, 1.4373,
        1.0699, 1.8489, 1.0358, 1.9720, 1.5125, 1.1718])


另外一个常用的函数就是item(), 它可以将一个标量Tensor转换成一个Python number：

In [6]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([-0.5278])
-0.5278484225273132


## 线性代数

另外，PyTorch还支持一些线性函数，这里提一下，免得用起来的时候自己造轮子，具体用法参考官方文档。如下表所示：

|函数	|功能|
|--|--|
|trace	|对角线元素之和(矩阵的迹)|
|diag	|对角线元素|
|triu/tril	|矩阵的上三角/下三角，可指定偏移量|
|mm/bmm	|矩阵乘法，batch的矩阵乘法|
|addmm/addbmm/addmv/addr/baddbmm..	|矩阵运算|
|t	|转置|
|dot/cross	|内积/外积|
|inverse	|求逆矩阵|
|svd	|奇异值分解|

这里列举一些常见的操作:

* 对Tensor求和/取平均

In [7]:
x = torch.ones(2, 3)
print(x)

print(x.sum())
print(x.sum(dim=0))
print(x.sum(dim=1))

print(x.mean())
print(x.mean(0))
print(x.mean(1))

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor(6.)
tensor([2., 2., 2.])
tensor([3., 3.])
tensor(1.)
tensor([1., 1., 1.])
tensor([1., 1.])


* 逐元素计算指数、对数

In [8]:
x = torch.tensor([1, 2.])
print(x)
print(torch.exp(x))
print(torch.log(x))

tensor([1., 2.])
tensor([2.7183, 7.3891])
tensor([0.0000, 0.6931])


* 逐元素乘积

In [9]:
x = torch.eye(2)
y = torch.ones(2, 2)
print(x)
print(y)
print(x*y)

tensor([[1., 0.],
        [0., 1.]])
tensor([[1., 1.],
        [1., 1.]])
tensor([[1., 0.],
        [0., 1.]])


* 矩阵乘法

In [10]:
torch.matmul(x, y)

tensor([[1., 1.],
        [1., 1.]])

PyTorch中的Tensor支持超过一百种操作，包括转置、索引、切片、数学运算、线性代数、随机数等等，可参考[官方文档](https://pytorch.org/docs/stable/tensors.html)。